In [1]:
from PIL import Image, ImageDraw, ImageFilter , ImageFont
import os
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!git clone https://github.com/cryptotreelab/NFT-art-generator.git imagess

Cloning into 'imagess'...
remote: Enumerating objects: 265, done.
remote: Counting objects: 100% (265/265), done.
remote: Compressing objects: 100% (260/260), done.
remote: Total 265 (delta 40), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (265/265), 432.27 MiB | 22.95 MiB/s, done.
Resolving deltas: 100% (40/40), done.
Checking out files: 100% (187/187), done.


In [3]:
def painter(base_image, path, photo_name, pos):    
# only png
    sec_path = path +'/'+photo_name
    try:
        sec_image = Image.open(sec_path)
    except :
        print('err in:', sec_path)
        
    back_image = base_image.copy()

    back_image.paste(sec_image,pos,sec_image) 
    
    back_image = back_image.convert('RGB')
    
    return back_image
    

In [4]:
def int_hash(hash ,a,b):
    return (int(hash,16)>>a)&b

In [5]:
# colors in display and interior
color_list_num = ['rgb(209,255,229)','rgb(142,9,0)','rgb(9,10,10)','rgb(41,1,53)','rgb(1,4,31)','rgb(40,0,1)','rgb(44,67,50)','rgb(9,10,10)','rgb(41,1,53)','rgb(1,4,31)','rgb(40,0,1)','rgb(44,67,50)'] 
color_list_display =  ['rgb(239,240,61)','rgb(255,241,14)','rgb(246,227,11)','rgb(21,15,185)','rgb(14,0,175)','rgb(229,21,161)','rgb(0,27,170)','rgb(151,39,0)','rgb(247,255,49)','rgb(247,255,49)','rgb(193,20,0)','rgb(91,0,0)']


In [6]:
#planet names that we used in Display
planet_name_list =["Alumina" ,
"Trippy Magma",
"Coppernicus",
"Kelvin",
"Tiffany Blue",
"Gem Crust",
"Ornge",
"Briliant Silver",
"Aurora",
"Amethyst",
"Graphene",
"Pomegrante",
"Yellow Cupcake",
"Cammo Lake",
"Duck Weed",
"Midas"] 

In [7]:

def make_scene(scene_width,scene_height,path,tokenId):
    img = Image.new("RGB", (scene_width, scene_height))
    # XmasBook Hash act as DNA for all of SpaceHooks
    hash_s = hex(int(hashdf.loc[tokenId%2022,"hash"],16) << (tokenId//2022)*64)

    for j in range(12):
        ###progress status
        # print("doing"+ str(tokenId) + ":"+ df.loc[j,'source'] , flush=True, end ="\r")
        
        ## pasting
        i = int_hash(hash_s,int(df.loc[j,'shift']),int(df.loc[j,'andNum']))%int(df.loc[j,'rem'])


        t_pos = (int(df.loc[j,'posx']) , int(df.loc[j,'posy']))
        
        t_path = path + '/'+df.loc[j,'source']

        entries = os.listdir(t_path)
        entries = sorted(entries)
        #some aritmatics that we used for making better paintings
        k= tokenId%5
        l = int(df.loc[j, "subSample"])
        k= tokenId%l
        f = i*l + k
        # print(f)
        img = painter(base_image=img ,path=t_path, photo_name=entries[f], pos=t_pos)
#     img.show()
#########################


    ksum = 0
    k = [1,1,1,1,1]
    for i in range(5):
        k[i] = int_hash(hash_s,220-i*4,15)+1
        ksum = k[i] + ksum
    a = ''
    for i in range(5):
        a = a + str(int(k[i]*100/ksum)).zfill(2) + ' '
        
    #draw object handles the texts on spaceship display and top left corner
    draw = ImageDraw.Draw(img)
    
   #draw0 : draw the token number at top left corner
    (x, y) = (20 , 20)
    tokenId_zero_filled = str(tokenId).zfill(4)
    hash_draw = tokenId_zero_filled #+ ": " + hash_story
    

    cb = int_hash(hash_s,int(df.loc[7,'shift']),int(df.loc[7,'andNum']))%int(df.loc[7,'rem'])

    planet_num = int_hash(hash_s,248,15)%16
    
    font_num = ImageFont.truetype("imagess/art-gen/PrintChar21.ttf", size=28)
    draw.text((x, y), hash_draw, fill=color_list_num[0], font=font_num)
    #draw1
    atm = "H  C  N  O  P \n" + a 
    font_atm = ImageFont.truetype("imagess/art-gen/PrintChar21.ttf", size=29)
    (x, y) = (928 , 48)

    tokenId_zero_filled = str(tokenId).zfill(4)
    hash_draw = tokenId_zero_filled #+ ": " + hash_story
    draw.text((x, y), atm, fill=color_list_display[cb], font=font_atm)
    
    #draw2
    (x, y) = (926 , 584)
    font_main_dis = ImageFont.truetype("imagess/art-gen/PrintChar21.ttf", size=26)
    main_dis = "Landing \nSuccessful\nPlanet:"
    draw.text((x, y), main_dis, fill=color_list_display[cb], font=font_main_dis)
    
    #draw3
    (x, y) = (926 , 664)
    font_main_dis = ImageFont.truetype("imagess/art-gen/PrintChar21.ttf", size=24)
    main_dis = '<'+planet_name_list[planet_num]+'>'
    draw.text((x, y), main_dis, fill=color_list_display[cb], font=font_main_dis)
########### image is ready
    return img 


In [8]:
def make_img_and_save_(tokenId,source_path,tar_path):

    img = make_scene(scene_width=3000,scene_height=1000,path=source_path,tokenId =tokenId)

    img_name = tar_path +'/'+str(tokenId)+'.jpeg'

    img.save(img_name)

In [9]:
path = 'imagess/source_Images/'
tar_path = '/content/drive/My Drive/target_data'
df = pd.read_csv('imagess/spacehook.csv')
hashdf = pd.read_csv('imagess/xmasbook_hash.csv')
# you may need to change this 
print(df.head())

        source  posx  posy  shift  andNum  rem  subSample
0          sky     0     0    252      15   12          4
1       planet     0     0    248      15   16          5
2  planet-line     0     0      0      15    1          1
3         moon     0     0    244      15    8          1
4    celestial     0     0    240      15    7          1


In [10]:
# entries = os.listdir(path+'/yuke')
# print(type(entries))
# print(sorted(entries))

In [15]:


number_of_tokens =4222 
for i in range(number_of_tokens):
    make_img_and_save_(tokenId=i,source_path= path,tar_path=tar_path)
    print("doing:"+ str(i)+'.jpeg')


KeyboardInterrupt: ignored

In [ ]:
make_img_and_save_(tokenId=6,source_path= path,tar_path=tar_path)


In [ ]:
!git clone https://github.com/cryptotreelab/NFT-art-generator.git imagess
from PIL import Image
# print(os.listdir)
# Image.open('images/Tree/oa1.png')

In [ ]:
Image.open('imagess/source_Images/Tree/oa1.png')

In [ ]:
print(os.listdir('imagess'))

In [ ]:
entries = os.listdir(t_path)